##Git and GitHub
Define access credentials and repo details.

The following environment variables (or secret keys) are assumed to be configured:
* GITHUB_USERNAME - github username
* GITHUB_USER_EMAIL - email configured for the github user
* GITHUB_TOKEN - github personal access token with relevant privileges (`repo` is enough)


In [1]:
from google.colab import userdata

github_user_email = userdata.get('GITHUB_USER_EMAIL')

In [3]:
token = userdata.get('GITHUB_TOKEN')
username = userdata.get('GITHUB_USERNAME')
repo = "mini-food-pic-classifier"

In [17]:
!git config --global user.name {username}
!git config --global user.email {github_user_email}
# !git config --global user.password ""

Get the latest code from the project repo.

Change the repo name and path as you need.

Create new repo in case you want to use a new repo.

In [4]:
!git clone https://{token}@github.com/{username}/{repo}

Cloning into 'mini-food-pic-classifier'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), done.


Change the working path to be inside the repo folder from this point and on.

In [5]:
%cd {repo}

/content/mini-food-pic-classifier


In [6]:
# project_name = "mini-food-pic-classifier"
project_path = "/content/" + repo

Currently assume the repo does not have the most updated version of this notebook, so copy it from the google drive:
* Connect to the drive
* Copy the latest notebook from the drive to its place in the local git repo

In [7]:
import os
load_notebooks_path = os.path.join(project_path, "notebooks")
if not os.path.exists(load_notebooks_path):
  os.mkdir(load_notebooks_path)

In [39]:
notebook_path = "\"/content/drive/MyDrive/Colab Notebooks/youtube/Learn PyTorch for deep learning in a day/05_pytorch_going_modular/pytorch_going_modular_cell_mode.ipynb\""

In [8]:
!cp  {notebook_path} ./notebooks/

In [9]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	notebooks/

nothing added to commit but untracked files present (use "git add" to track)


##0. Running a notebook in cell mode

This notebook is intended to run normally, one cell at a time.

## 1. Get data
We're going to start by downloading the `pizza_steak_sushi` datast with images of pizza, steak and sushi.

In [29]:
import os
import zipfile

from pathlib import Path

import requests

# Setup path to data folder
data_path = Path(os.path.join(project_path, "data/"))
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Did not find /content/mini-food-pic-classifier/data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


In [30]:
# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('/content/mini-food-pic-classifier/data/pizza_steak_sushi/train'),
 PosixPath('/content/mini-food-pic-classifier/data/pizza_steak_sushi/test'))

###Update Git

In [10]:
!git checkout -b cell_mode_get_data

Switched to a new branch 'cell_mode_get_data'


In [19]:
!git add notebooks
!git commit -m "Get data in cell mode"

[cell_mode_get_data 06fef79] Get data in cell mode
 1 file changed, 1 insertion(+)
 create mode 100644 notebooks/pytorch_going_modular_cell_mode.ipynb


In [20]:
!git status

On branch cell_mode_get_data
nothing to commit, working tree clean


In [22]:
!git checkout main
!git merge cell_mode_get_data
!git push origin main

Everything up-to-date


##2. Create Datasets and DataLoaders
Let's turn our data into PyTorch `Dataset`'s and `DataLoader`'s and find out a few useful attributes from them such as `classes` and their lengths.

In [31]:
from torchvision import datasets, transforms

# Create simple transform
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Use ImageFolder to create dataset(s)
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 225
    Root location: /content/mini-food-pic-classifier/data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 75
    Root location: /content/mini-food-pic-classifier/data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [32]:
# Get class names as a list
class_names = train_data.classes
class_names

['pizza', 'steak', 'sushi']

In [33]:
# Can also get class names as a dict
class_dict = train_data.class_to_idx
class_dict

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [34]:
# Check the lengths
len(train_data), len(test_data)

(225, 75)

In [35]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=1, # how many samples per batch?
                              num_workers=1, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=1,
                             num_workers=1,
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7ba6fb41b190>,
 <torch.utils.data.dataloader.DataLoader at 0x7ba6fb41b100>)

In [36]:
# Check out single image size/shape
img, label = next(iter(train_dataloader))

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([1, 3, 64, 64]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([1])


###Update Git

In [41]:
!cp {notebook_path} ./notebooks/
!git checkout -b create_datasets_and_dataloaders

M	notebooks/pytorch_going_modular_cell_mode.ipynb
Already on 'create_datasets_and_dataloaders'


In [42]:
!git add notebooks/pytorch_going_modular_cell_mode.ipynb
!git commit -m "Create Datasets and DataLoaders"

[create_datasets_and_dataloaders bec1031] Create Datasets and DataLoaders
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite notebooks/pytorch_going_modular_cell_mode.ipynb (75%)
Everything up-to-date


In [44]:
!git checkout main
!git merge create_datasets_and_dataloaders
!git push origin main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating 4a19b62..bec1031
Fast-forward
 notebooks/pytorch_going_modular_cell_mode.ipynb | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 3.20 KiB | 1.60 MiB/s, done.
Total 8 (delta 4), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (4/4), completed with 2 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   4a19b62..bec1031  main -> main


##3. Making a model (TinyVGG)
We're going to use the TinyVGG model from the CNN Explainer website.


In [45]:
import torch

from torch import nn

class TinyVGG(nn.Module):
    """Creates the TinyVGG architecture.

    Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from?
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
        )

    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x
        # return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion

Now let's create an instance of TinyVGG and put it on the target device.

>Note: If you're using Google Colab, and you'd like to use a GPU (recommended), you can turn one on via going to Runtime -> Change runtime type -> Hardware accelerator -> GPU.

In [46]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate an instance of the model
torch.manual_seed(42)
model_0 = TinyVGG(input_shape=3, # number of color channels (3 for RGB)
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)
model_0

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)


Let's check out our model by doing a dummy forward pass.

In [47]:
# 1. Get a batch of images and labels from the DataLoader
img_batch, label_batch = next(iter(train_dataloader))

# 2. Get a single image from the batch and unsqueeze the image so its shape fits the model
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image shape: {img_single.shape}\n")

# 3. Perform a forward pass on a single image
model_0.eval()
with torch.inference_mode():
    pred = model_0(img_single.to(device))

# 4. Print out what's happening and convert model logits -> pred probs -> pred label
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image shape: torch.Size([1, 3, 64, 64])

Output logits:
tensor([[ 0.0208, -0.0020,  0.0095]])

Output prediction probabilities:
tensor([[0.3371, 0.3295, 0.3333]])

Output prediction label:
tensor([0])

Actual label:
0


###Update Git

In [49]:
!cp {notebook_path} ./notebooks/
!git checkout -b add_model

Switched to a new branch 'add_model'


In [51]:
!git add notebooks/pytorch_going_modular_cell_mode.ipynb
!git commit -m "add model to notebook"

[add_model 91e4bf7] add model to notebook
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite notebooks/pytorch_going_modular_cell_mode.ipynb (66%)


In [52]:
!git checkout main
!git merge add_model
!git push origin main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating bec1031..91e4bf7
Fast-forward
 notebooks/pytorch_going_modular_cell_mode.ipynb | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 3.14 KiB | 3.14 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   bec1031..91e4bf7  main -> main


##4. Creating `train_step()` and `test_step()` functions and `train()` to combine them
Let's start by making `train_step()`.

In [53]:
from typing import Tuple

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

Now we'll do `test_step()`.

In [54]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

And we'll combine `train_step()` and `test_step()` into `train()`.

In [55]:
from typing import Dict, List

from tqdm.auto import tqdm

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]}
    For example if training for epochs=2:
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results

###Update Git

In [56]:
!cp {notebook_path} ./notebooks/
!git checkout -b add_train_test_functions

Switched to a new branch 'add_train_test_functions'


In [57]:
!git add notebooks/pytorch_going_modular_cell_mode.ipynb
!git commit -m "add train_step test_step and train functions"

[add_train_test_functions 535dd4b] add train_step test_step and train functions
 1 file changed, 1 insertion(+), 1 deletion(-)


In [58]:
!git checkout main
!git merge add_train_test_functions
!git push origin main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating 91e4bf7..535dd4b
Fast-forward
 notebooks/pytorch_going_modular_cell_mode.ipynb | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 3.02 KiB | 3.02 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   91e4bf7..535dd4b  main -> main


##5. Creating a function to save the model
Let's setup a function to save our model to a directory.

In [59]:
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.

    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

###Update Git

In [60]:
!cp {notebook_path} ./notebooks/
!git checkout -b add_save_model_function

Switched to a new branch 'add_save_model_function'


In [61]:
!git add notebooks/pytorch_going_modular_cell_mode.ipynb
!git commit -m "add function to save the model"

[add_save_model_function e778967] add function to save the model
 1 file changed, 1 insertion(+), 1 deletion(-)


In [62]:
!git checkout main
!git merge add_save_model_function
!git push origin main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating 535dd4b..e778967
Fast-forward
 notebooks/pytorch_going_modular_cell_mode.ipynb | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.34 KiB | 1.34 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   535dd4b..e778967  main -> main


##6. Train, evaluate and save the model
Let's leverage the functions we've got above to train, test and save a model to file.

In [66]:
# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 5

# Recreate an instance of TinyVGG
model_0 = TinyVGG(input_shape=3, # number of color channels (3 for RGB)
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_0_results = train(model=model_0,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS,
                        device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# Save the model
save_model(model=model_0,
           target_dir="models",
           model_name="05_going_modular_cell_mode_tinyvgg_model.pth")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0923 | train_acc: 0.3911 | test_loss: 1.0727 | test_acc: 0.4133
Epoch: 2 | train_loss: 1.0273 | train_acc: 0.5156 | test_loss: 1.0164 | test_acc: 0.4267
Epoch: 3 | train_loss: 0.9614 | train_acc: 0.5111 | test_loss: 0.9927 | test_acc: 0.4667
Epoch: 4 | train_loss: 0.9160 | train_acc: 0.5733 | test_loss: 0.9778 | test_acc: 0.4133
Epoch: 5 | train_loss: 0.8873 | train_acc: 0.6089 | test_loss: 0.9971 | test_acc: 0.5333
[INFO] Total training time: 17.736 seconds
[INFO] Saving model to: models/05_going_modular_cell_mode_tinyvgg_model.pth


###Update Git

In [63]:
!cp {notebook_path} ./notebooks/
!git checkout -b train_evaluate_save_model

Switched to a new branch 'train_evaluate_save_model'


In [64]:
!git add notebooks/pytorch_going_modular_cell_mode.ipynb
!git commit -m "train, evaluate and save the model"

[train_evaluate_save_model 0953c71] train, evaluate and save the model
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite notebooks/pytorch_going_modular_cell_mode.ipynb (97%)


In [65]:
!git checkout main
!git merge train_evaluate_save_model
!git push origin main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating e778967..0953c71
Fast-forward
 notebooks/pytorch_going_modular_cell_mode.ipynb | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.41 KiB | 1.41 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   e778967..0953c71  main -> main


###Update Git

In [ ]:
!cp {notebook_path} ./notebooks/
!git checkout -b run_train_evaluate_save_model

Switched to a new branch 'train_evaluate_save_model'


In [ ]:
!git add notebooks/pytorch_going_modular_cell_mode.ipynb
!git commit -m "run train, evaluate and save the model"

[train_evaluate_save_model 0953c71] train, evaluate and save the model
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite notebooks/pytorch_going_modular_cell_mode.ipynb (97%)


In [ ]:
!git checkout main
!git merge run_train_evaluate_save_model
!git push origin main

Switched to branch 'main'
Your branch is up to date with 'origin/main'.
Updating e778967..0953c71
Fast-forward
 notebooks/pytorch_going_modular_cell_mode.ipynb | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.41 KiB | 1.41 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/guyfuchs/mini-food-pic-classifier
   e778967..0953c71  main -> main
